<a href="https://colab.research.google.com/github/hyej987/TECHROSS/blob/master/DATA_PLOT/DATA_CONTROL_CHART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지로딩

In [1]:
from google.colab import drive
drive.mount('/content/gdrive') 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 데이터 로딩

In [2]:
DATA=pd.read_csv('/content/gdrive/MyDrive/DB/DATA.csv')
OPTIME=pd.read_csv('/content/gdrive/MyDrive/DB/OPTIME.csv')
DATA['KEY'] = DATA['SHIP_ID'] + "_" + DATA['SECTION'].astype(str) + "_" + DATA['OP_INDEX'].astype(str)
OPTIME['KEY'] = OPTIME['SHIP_ID'] + "_" + OPTIME['SECTION'].astype(str) + "_" + OPTIME['OP_INDEX'].astype(str)
DATA_JOIN_OPTIME = pd.merge(DATA,OPTIME,how = 'left', on = 'KEY')
DATA_JOIN_OPTIME = DATA_JOIN_OPTIME.drop(columns = ['SHIP_ID_y', 'SYSTEM_y', 'OP_INDEX_y','SECTION_y'])
DATA_JOIN_OPTIME = DATA_JOIN_OPTIME.drop(columns = ['KEY'])
DATA_JOIN_OPTIME=DATA_JOIN_OPTIME.rename(columns = {'OP_INDEX_x' : 'OP_INDEX', 'SHIP_ID_x' : 'SHIP_ID', 'SYSTEM_x' : 'SYSTEM', 'SECTION_x' : 'SECTION'})
SHIP_INFO = pd.read_csv('/content/gdrive/MyDrive/DB/SHIPINFO.csv')
SHIP_INFO=SHIP_INFO[["ship_id","ship_name"]]
SHIP_INFO.rename(columns = {"ship_id":"SHIP_ID","ship_name":"SHIP_NAME"},inplace = True)
DATA_JOIN_OPTIME=pd.merge(DATA_JOIN_OPTIME,SHIP_INFO, how = 'left' , on = 'SHIP_ID')
DATA_JOIN_OPTIME=DATA_JOIN_OPTIME.dropna(subset = ["OP_TYPE"])
DATA_JOIN_OPTIME=DATA_JOIN_OPTIME.dropna(subset = ["LINE"])
DATA_JOIN_OPTIME["SECTION"]=DATA_JOIN_OPTIME["SECTION"].astype(int)
DATA_JOIN_OPTIME["OP_TYPE"]=DATA_JOIN_OPTIME["OP_TYPE"].astype(int)
DATA_JOIN_OPTIME["LINE"]=DATA_JOIN_OPTIME["LINE"].astype(int)
DATA_JOIN_OPTIME=DATA_JOIN_OPTIME.loc[DATA_JOIN_OPTIME['SHIP_ID'] != 'Unknown',:]
DATA_JOIN_OPTIME=DATA_JOIN_OPTIME[['SHIP_NAME','SHIP_ID','SECTION','OP_TYPE','LINE','CSU', 'STS', 'FTS', 'FMU', 'TRO', 'ANU', 'RATE', 'CURRENT', 'VOLTAGE','DATA_TIME']]
DATA_JOIN_OPTIME["KEY"]=DATA_JOIN_OPTIME['SHIP_ID'] +"_" +DATA_JOIN_OPTIME["SECTION"].astype(str) +"_"+ DATA_JOIN_OPTIME["OP_TYPE"].astype(str) + "_" + DATA_JOIN_OPTIME["LINE"].astype(str)

# 플랏로딩

In [4]:
list_data = ['CSU', 'STS', 'FTS', 'FMU', 'TRO', 'ANU', 'RATE', 'CURRENT', 'VOLTAGE']
data_key = DATA_JOIN_OPTIME['KEY'].unique()
for i in range(len(data_key)) : 
  for j in range(len(list_data)) : 
    data = DATA_JOIN_OPTIME.loc[(DATA_JOIN_OPTIME["KEY"] == data_key[i]) ,:]
    data = data.reset_index().drop(columns = ["index"])
    data= data.drop_duplicates(subset = ["DATA_TIME"])
    data=data.reset_index().drop(columns = ["index"])
    ship_name=data.iloc[0,0]
    section=data.iloc[0,2]
    op_type=data.iloc[0,3]
    line=data.iloc[0,4]
    sigma3=data[f'{list_data[j]}'].std() * 3
    mean = data[f'{list_data[j]}'].mean()
    fig = plt.figure(figsize = (30,8))
    ax = fig.subplots()
    plt.plot(data[f'{list_data[j]}'], marker = ".")
    plt.xticks(rotation = 90)
    plt.axhline(mean - sigma3, color = "r")
    plt.axhline(mean + sigma3, color = "r")
    plt.title(f"{list_data[j]}"+"     "+"SHIP_NAME : "+ f"{ship_name}"+"      "+ "SECTION :"+ f"{section}"+"     " + "OP_TYPE : "+ f"{op_type}"+ "     "+ "LINE : "+f"{line}"+ "     "+ 
          "LCL : " + f"{np.round(mean-sigma3)}" +"     " +"UCL : " + f"{np.round(mean+sigma3)}", size = 20)
    ax.xaxis.set_ticks([])
    plt.savefig("/content/gdrive/MyDrive/TECHCROSS 자료파일/수치형 자료분석(lcl,ucl)/DATA/" + f"{list_data[j]}"+" "+"SHIP_NAME : "+ f"{ship_name}"+" "+ "SECTION :"+ f"{section}"+" " + "OP_TYPE : "+ f"{op_type}"+ " "+ "LINE : "+f"{line}", format = 'png')